In [1]:
import sys
import time
import torch
from torch.backends import cudnn
from matplotlib import colors
import cv2
import numpy as np
import os
import statistics
from skimage import io, draw
from yolox.data.datasets import COCO_CLASSES
from yolox.utils import fuse_model, get_model_info, postprocess, vis
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from loguru import logger
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
import yolo_detect as yd


#바운딩박스 그려주는 함수         
def vis(img, boxes, scores, conf = 0.5):
    for i in range(len(boxes)):
        box = boxes[i]
        score = scores[i]
        if score < conf:
            continue
        x0 = int(box[0])
        y0 = int(box[1])
        x1 = int(box[2])
        y1 = int(box[3])

        color = (0, 0, 255)
        text = '{}:{:.1f}%'.format(f'NO', score * 100)
        txt_color = (255, 255, 255)
        font = cv2.FONT_HERSHEY_SIMPLEX

        txt_size = cv2.getTextSize(text, font, 0.6, 1)[0]
        cv2.rectangle(img, (x0, y0), (x1, y1), color, 2)

        txt_bk_color = (0, 0, 0)
        cv2.rectangle(
            img,
            (x0, y0 + 1),
            (x0 + txt_size[0] + 1, y0 + int(1.5*txt_size[1])),
            txt_bk_color,
            -1
        )
        cv2.putText(img, text, (x0, y0 + txt_size[1]), font, 0.4, txt_color, thickness=1)

    return img

#클래스 (차량, 오토바이 등) 분류해주고 박스 원래 사이즈로 돌려놓기     
def visual(output, img_info, zone,cls_conf=0.5):
    zone_xmin = int(zone[0])
    zone_ymin = int(zone[1])
    zone_xmax = int(zone[2])
    zone_ymax = int(zone[3])
    

    ratio = img_info["ratio"]
    
    #아무런 output이 나오지 않을경우 
    if output is None:
        return None, None, None
    output = output.cpu()
    bboxes = output[:, 0:4]
    # preprocessing: resize
    bboxes /= ratio
    cls = output[:, 6] #현재 이미지에서 잡힌 객체들의 번호 차 = 2번 , 신호등 = 9 번 
    scores = output[:, 4] * output[:, 5] #잡힌 객체들의 신뢰도 수준 
    original_bboxes = []
    original_cls = []
    original_scores = []
    for i in range(len(bboxes)):
        box = bboxes[i]
        cls_id = int(cls[i])
        score = scores[i]
        
        x_min = int(box[0])
        y_min = int(box[1])
        x_max = int(box[2])
        y_max = int(box[3])

        #차, 오토바이, 버스, 트럭에 대해서만 박스랑 스코어 생성해서 트래킹에 보내버리기 
        if cls_id in [2,3,5,7] and cls_conf < score :
            #특정 zone에 있는 것들만 박스로 내보내기 
            c = (zone_ymin <= y_max and y_max < zone_ymax) and (zone_xmin <= x_max and x_max < zone_xmax)
            d = (zone_ymin <= y_max and y_max < zone_ymax) and (zone_xmin <= x_min and x_min < zone_xmax)
            if c and d : 
                print('차 지나간다아아아')
                original_cls.append(cls_id)
                original_scores.append(score)
                original_bboxes.append([x_min,y_min,x_max,y_max])

    #output은 나왔지만 차량에 해당 하는 output이 없을경우 (사람만 포착이 되었을경우)     
    if len(original_bboxes) == 0 :
        return None, None, None
    
#     original_bboxes = torch.Tensor(original_bboxes)
#     original_cls = torch.Tensor(original_cls)
#     original_scores = torch.Tensor(original_scores)

    return original_bboxes, original_cls, original_scores

#비디오 읽고, 전체적인 코드 실행 
def proc(video_path,output_file):
    det = yd.YoloDetector_st() #detector 소환      
    det.load() #모델 load
    cap = cv2.VideoCapture(video_path)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  # float
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float
    fps = cap.get(cv2.CAP_PROP_FPS)
    print('fps',fps)
    save_path = output_file
    logger.info(f"video save_path is {save_path}")
    vid_writer = cv2.VideoWriter(
        save_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (int(width), int(height))
    )

    cnt = 0
    if cap.isOpened() :    
        while True:
            cnt+=1
            #원본이미지 = frame 
            ret, frame = cap.read()
            if ret == False :
                break
            if cnt < 0:
                continue
            #프레임수 특정값 이상일떄는 break 할때 
#             if cnt > 10:
#                 break
            #현재 프레임
            num_frame = str(round(cap.get(cv2.CAP_PROP_POS_FRAMES)))
            print('frame : ',num_frame)
            #detect 완료후 결과 받기
            outputs, img_info = det.inference(frame,num_frame)
            #특정 클래스만 받아오기 
            bboxes, class_id, scores = visual(outputs[0], img_info,zone,det.confthre)
            #박스 그리기 
            if not bboxes == None :
                result_frame = vis(frame, bboxes, scores,conf = 0.5)
            else : result_frame = frame 
            
            vid_writer.write(result_frame)
            ch = cv2.waitKey(1)
            if ch == 27 or ch == ord("q") or ch == ord("Q"):
                break

# zone = [zone_xmin, zone_ymin,zone_xmax,zone_ymax] 
zone = [1025, 875,1530,970]

input_list = [
    './input_ST/1.mp4',
#     './input/2.mp4',
#     './input/3.mp4',
#     './input/4.mp4',
#     './input/5.mp4',
#     './input/6.mp4',
#     './input/7.mp4',
#     './input/8.mp4',
#     './input/9.mp4',
#     './input/10.mp4',
#     './input/11.mp4',
#     './input/12.mp4',
]
output_list = [
    './output/1.mp4',
#     './output/2.mp4',
#     './output/3.mp4',
#     './output/4.mp4',
#     './output/5.mp4',
#     './output/6.mp4',
#     './output/7.mp4',
#     './output/8.mp4',
#     './output/9.mp4',
#     './output/10.mp4',
#     './output/11.mp4',
#     './output/12.mp4',
]


for idx, input_path in enumerate(input_list):
    print(f"{input_path} start!!")
    proc(input_list[idx],output_list[idx])
    print(f'####################{input_list[idx]} complete#########################')

2022-04-13 09:36:03.560 | INFO     | yolo_detect:load:143 - Model Summary: Params: 8.97M, Gflops: 26.81


./input_ST/1.mp4 start!!


2022-04-13 09:36:06.461 | INFO     | __main__:proc:119 - video save_path is ./output/1.mp4


fps 28.0
frame :  1
frame :  2
frame :  3
frame :  4
frame :  5
frame :  6
frame :  7
frame :  8
frame :  9
frame :  10
frame :  11
frame :  12
frame :  13
frame :  14
frame :  15
frame :  16
frame :  17
frame :  18
frame :  19
frame :  20
frame :  21
frame :  22
frame :  23
frame :  24
frame :  25
frame :  26
frame :  27
frame :  28
frame :  29
frame :  30
frame :  31
frame :  32
frame :  33
frame :  34
frame :  35
frame :  36
frame :  37
frame :  38
frame :  39
frame :  40
frame :  41
frame :  42
frame :  43
frame :  44
frame :  45
frame :  46
frame :  47
frame :  48
frame :  49
frame :  50
frame :  51
frame :  52
frame :  53
frame :  54
frame :  55
frame :  56
frame :  57
frame :  58
frame :  59
frame :  60
frame :  61
frame :  62
frame :  63
frame :  64
frame :  65
frame :  66
frame :  67
frame :  68
frame :  69
frame :  70
frame :  71
frame :  72
frame :  73
frame :  74
frame :  75
frame :  76
frame :  77
frame :  78
frame :  79
frame :  80
frame :  81
frame :  82
frame :  83
fram